In [ ]:
import os
import numpy as np
import pandas as pd
import torch
# import torch.nn.functional as F
from pyspark.sql import functions as F
import torch_geometric
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col


spark = SparkSession.builder \
    .appName("TransactionClassificationGNN") \
    .getOrCreate()

spark.conf.set("spark.sql.debug.maxToStringFields", "100")

RANDOM_STATE = 42
torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

class SparkGNN:
    def __init__(self, input_dir, num_features=166, num_classes=2):
        """
        Initialize the Spark-PyTorch Geometric Pipeline
        
        :param input_dir: Directory containing input CSV files
        :param num_features: Number of input features
        :param num_classes: Number of output classes
        """
        self.input_dir = input_dir
        self.num_features = num_features
        self.num_classes = num_classes
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
    def load_data(self):
        """
        Load data using PySpark
        
        :return: Tuple of features, classes, and edgelist DataFrames
        """
        # Load CSV files
        features_path = os.path.join(self.input_dir, 'elliptic_txs_features.csv')
        classes_path = os.path.join(self.input_dir, 'elliptic_txs_classes.csv')
        edgelist_path = os.path.join(self.input_dir, 'elliptic_txs_edgelist.csv')
        
        # Read as Spark DataFrames
        features_df = spark.read.csv(features_path, header=False)
        classes_df = spark.read.csv(classes_path, header=True)
        edgelist_df = spark.read.csv(edgelist_path, header=True)
        
        # Rename and prepare features columns
        features_cols = ['txId'] + [f'V{i}' for i in range(1, self.num_features + 1)]
        features_df = features_df.toDF(*features_cols)
        
        # Map classes
        classes_df = classes_df.withColumn('class_mapped', 
            F.when(F.col('class') == '1', 'illicit')
            .when(F.col('class') == '2', 'licit')
            .otherwise('unknown')
        )
        
        return features_df, classes_df, edgelist_df
    
    def preprocess_data(self, features_df, classes_df):
        """
        Preprocess data using PySpark ML Pipeline
        
        :param features_df: Features DataFrame
        :param classes_df: Classes DataFrame
        :return: Preprocessed DataFrame
        """
        # Prepare feature columns for vector assembly
        feature_cols = [f'V{i}' for i in range(1, self.num_features + 1)]
        
        # Vector Assembler to combine features
        assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
        
        # String Indexer for target variable
        indexer = StringIndexer(inputCol='class_mapped', outputCol='label')
        
        # Create ML Pipeline
        pipeline = Pipeline(stages=[assembler, indexer])
        
        # Fit and transform data
        preprocessor = pipeline.fit(features_df.join(classes_df, 'txId'))
        processed_df = preprocessor.transform(features_df.join(classes_df, 'txId'))
        
        return processed_df

    
    def create_graph_data(self, processed_df, edgelist_df):
        """
        Create PyTorch Geometric graph data
        
        :param processed_df: Preprocessed DataFrame
        :param edgelist_df: Edgelist DataFrame
        :return: PyTorch Geometric Data object
        """
        # Create tx_id to index mapping
        tx_id_mapping = {row['txId']: idx for idx, row in enumerate(processed_df.collect())}
        
        # Prepare node features
        node_features = torch.tensor(
            processed_df.select('features').rdd.map(lambda x: x['features'].toArray()).collect(), 
            dtype=torch.float
        )
        
        # Prepare node labels
        node_labels = torch.tensor(
            processed_df.select('label').rdd.map(lambda x: x['label']).collect(), 
            dtype=torch.long
        )
        
        # Prepare edge index
        edge_df = edgelist_df.filter(
            F.col('txId1').isin(list(tx_id_mapping.keys())) & 
            F.col('txId2').isin(list(tx_id_mapping.keys()))
        )
        
        edge_index_list = [
            (tx_id_mapping[row['txId1']], tx_id_mapping[row['txId2']]) 
            for row in edge_df.collect()
        ]
        
        edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
        
        # Create PyTorch Geometric Data object
        data = Data(x=node_features, edge_index=edge_index, y=node_labels)
        
        # Split data
        num_nodes = data.num_nodes
        train_mask = torch.zeros(num_nodes, dtype=torch.bool)
        val_mask = torch.zeros(num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(num_nodes, dtype=torch.bool)
        
        # Randomly split known nodes
        known_mask = (data.y == 0) | (data.y == 1)
        known_indices = torch.nonzero(known_mask).squeeze()
        
        # Shuffle known indices
        perm = torch.randperm(len(known_indices))
        known_indices = known_indices[perm]
        
        # Split ratios
        train_ratio, val_ratio = 0.8, 0.1
        train_size = int(train_ratio * len(known_indices))
        val_size = int(val_ratio * len(known_indices))
        
        # Create masks
        train_indices = known_indices[:train_size]
        val_indices = known_indices[train_size:train_size+val_size]
        test_indices = known_indices[train_size+val_size:]
        
        train_mask[train_indices] = True
        val_mask[val_indices] = True
        test_mask[test_indices] = True
        
        data.train_mask = train_mask
        data.val_mask = val_mask
        data.test_mask = test_mask
        
        return data.to(self.device)
    
    class GCN(torch.nn.Module):
        def __init__(self, num_features, num_classes):
            super().__init__()
            self.conv1 = GCNConv(num_features, 16)
            self.conv2 = GCNConv(16, num_classes)
        
        def forward(self, data):
            x, edge_index = data.x, data.edge_index
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
    
    def train(self, data, num_epochs=100):
        """
        Train the Graph Neural Network
        
        :param data: PyTorch Geometric Data object
        :param num_epochs: Number of training epochs
        :return: Trained model and training metrics
        """
        # Initialize model
        model = self.GCN(num_features=data.num_features, num_classes=self.num_classes).to(self.device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)
        criterion = torch.nn.CrossEntropyLoss()
        
        # Training loop
        model.train()
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            
            # Print progress
            if epoch % 10 == 0:
                print(f'Epoch {epoch:03d}, Loss: {loss.item():.4f}')
        
        return model
    
    def evaluate(self, model, data):
        """
        Evaluate the model
        
        :param model: Trained PyTorch Geometric model
        :param data: PyTorch Geometric Data object
        :return: Evaluation metrics
        """
        model.eval()
        with torch.no_grad():
            out = model(data)
            pred = out.argmax(dim=1)
            
            # Metrics for test set
            test_mask = data.test_mask
            y_true = data.y[test_mask].cpu().numpy()
            y_pred = pred[test_mask].cpu().numpy()
            
            # Calculate metrics
            precision = precision_score(y_true, y_pred, average='weighted')
            recall = recall_score(y_true, y_pred, average='weighted')
            f1 = f1_score(y_true, y_pred, average='weighted')
            
            # Print classification report
            print("\nClassification Report:")
            print(classification_report(y_true, y_pred, target_names=['illicit', 'licit']))
            
            return {
                'precision': precision,
                'recall': recall,
                'f1_score': f1
            }
    
    def run_pipeline(self):
        """
        Run full ML pipeline
        """
        # Load data
        features_df, classes_df, edgelist_df = self.load_data()
            # Convert string columns to numeric
        # def convert_to_numeric(df):
        #     # List of columns to convert
        #     numeric_cols = ['V162', 'V163', 'V164', 'V165', 'V166']
            
        #     # Convert each column to double
        #     for column in numeric_cols:
        #         df = df.withColumn(column, col(column).cast(DoubleType()))
            
        #     return df
        
        # Before creating the pipeline, convert columns
        numeric_cols = [col_name for col_name in features_df.columns if col_name != 'txId']
        for column in numeric_cols:
            features_df = features_df.withColumn(column, col(column).cast(DoubleType()))
        
        # Preprocess data
        processed_df = self.preprocess_data(features_df, classes_df)
        
        # Create graph data
        graph_data = self.create_graph_data(processed_df, edgelist_df)
        
        # Train model
        model = self.train(graph_data)
        
        # Evaluate model
        metrics = self.evaluate(model, graph_data)
        
        return model, metrics

# Main execution
if __name__ == '__main__':
    # Specify input directory containing CSV files
    input_directory = ''
    
    # Initialize and run pipeline
    gnn_pipeline = SparkGNN(input_directory)
    model, metrics = gnn_pipeline.run_pipeline()
    
    # Print final metrics
    print("\nFinal Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")
    
    # Close Spark session
    spark.stop()